# Reproduce the porous opacities from Kataoka et al. 2014

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import dsharp_opac as do
%matplotlib inline

Select the porosity values

In [ ]:
porosities = [0, 0.9, 0.99]

# select opacities:
# - 'dsharp': the DHSHARP opacity mix
# - 'kataoka': similar constants as Kataoka+2014
# - 'kataoka_orig': digitized values from Kataoka+2014

opac_type = 'kataoka_orig'

Get optical constants for the DSHARP mix:

In [ ]:
if opac_type == 'dsharp':
    constants = [do.get_dsharp_mix(porosity=porosity) for porosity in porosities]

Reproduce Kataoka 2014 opacities:

In [ ]:
if opac_type == 'kataoka':

    c0  = do.diel_vacuum()
    c1  = do.diel_draine2003('astrosilicates')
    c2  = do.diel_henning('organics', new=False)
    c3  = do.diel_warren84()

    c_array   = [c0,  c1,      c2,      c3]
    m_fract   = [0,   2.64e-3, 3.53e-3, 5.55e-3]
    densities = [0.0, 3.50,    1.50,    0.92]
    vol_fract = [0.0, 0.08,    0.26,    0.66]

    rho_s = (np.array(vol_fract)*densities).sum()

    # Extrapolate the optical constants

    c1.extrapolate_constants_up(0.05,1e5, kind='linear')
    c2.extrapolate_constants_up(1, 1e5, kind='linear')
    c3.extrapolate_constants_up(100, 1e5, kind='linear')

    mix1 = do.diel_mixed(c_array, vol_fract, rule='Bruggeman')
    
    constants = [[do.diel_mixed([do.diel_vacuum(), mix1], [porosity, 1 - porosity], rule='Maxwell-Garnett'), rho_s * (1 - porosity)] for porosity in porosities]
    
    # compare to Kataoka
    mix_k = do.diel_from_lnk_file(do.get_datafile('kataoka_mix.lnk'), headerlines=5)
    mix_k.material_str = 'Kataoka et al. 2014'
    
    do.compare_nk([mix_k, mix1])

Use the digitized optical constants. Note: we need to extrapolate to smaller sizes

In [ ]:
if opac_type == 'kataoka_orig':
    rho_s = 1.68 # value from paper, but should perhaps be 1.277 if vol. fractions and densities from paper are used
    mix_k = do.diel_from_lnk_file(do.get_datafile('kataoka_mix.lnk'), headerlines=5)
    mix_k.material_str = 'Kataoka et al. 2014'
    
    mix_ke = do.diel_from_lnk_file(do.get_datafile('kataoka_mix.lnk'), headerlines=5)
    mix_ke.material_str = 'Kataoka et al. 2014'
    mix_ke.extrapolate_constants_down(1e-5, 1.5e-4, kind='linear')

    do.compare_nk([mix_k, mix_ke])
    
    
    constants = [[do.diel_mixed([do.diel_vacuum(), mix_ke], [porosity, 1 - porosity], rule='Maxwell-Garnett'), rho_s * (1 - porosity)] for porosity in porosities]

Use the digitiezed data from Kataoka paper.

Define size, mass, and wavelength grid. 

In [ ]:
a   = np.logspace(-5, 3, 200)
lam = np.logspace(-5, 0, 300) # np.logspace(-4, 0, 60)

Calculate the opacities

In [ ]:
results = [do.get_opacities(a, lam, rho_s, oc, extrapolate_large_grains=True) for oc,rho_s in constants]

### Reproduce [Kataoka et al. 2014, Fig. 2 b&d](https://www.aanda.org/articles/aa/full_html/2014/08/aa23199-13/F2.html)

In [ ]:
kabs_kf2b = np.loadtxt(do.get_datafile('Kataoka2014-Fig2b-1e-5cm.csv'), delimiter=',')
kabs_kf2d = np.loadtxt(do.get_datafile('Kataoka2014-Fig2d-1e-5cm.csv'), delimiter=',')


fig, ax = plt.subplots(2, 1, figsize=(8, 12), sharex=True, sharey=True)
a_plot = np.logspace(-5, -1, 5)

ax = ax.ravel()

i_res = 0
for i in range(len(a_plot)):
    if i == 0:
        lw = 3
    else:
        lw = 1
    ia = results[i_res]['a'].searchsorted(a_plot[i])
    ax[0].loglog(results[i_res]['lam'], results[i_res]['k_abs'][ia, :], lw=lw, label='$f = $ {:g}, $a =$ {:.1e}'.format(1 - porosities[i_res], results[i_res]['a'][ia]))
ax[0].loglog(kabs_kf2b[:, 0], kabs_kf2b[:, 1], 'k--', lw=1, label='Kataoka et al. 2014')
    
    
a0 = 0.1e-4    
for i, res in enumerate(results):
    ia = res['a'].searchsorted(a0 / (1-porosities[i]))
    ax[1].loglog(res['lam'], res['k_abs'][ia, :], label='$a = $ {:.1e}, $a\cdot f = ${:.4g}'.format(res['a'][ia], res['a'][ia] * (1 - porosities[i])))
ax[1].loglog(kabs_kf2d[:, 0], kabs_kf2d[:, 1], 'k--', lw=1, label='Kataoka et al. 2014')

for _ax in ax:
    _ax.legend().get_frame().set_alpha(0)
    _ax.set_ylabel('$\kappa_\mathrm{abs}$ [cm$^2$ g$^{-1}$]')

ax[0].set_xlim(1e-4, 1e1)
ax[0].set_ylim(1e-4, 1e5)
ax[1].set_xlabel('$\lambda$ [cm]')

fig.subplots_adjust(hspace=0.05)
fig.savefig('Kataoka2014_F2b.pdf', transparent=True)

### Size average and reproduce [Fig. 11](https://www.aanda.org/articles/aa/full_html/2014/08/aa23199-13/F11.html)

Note: they use a size distribution of $n(a)\propto a^{-2}$, not the usual MRN.

In [ ]:
averages = [do.size_average_opacity([0.1, 0.3], a, lam, res['k_abs'], res['k_sca'], q=2, plot=False) for res in results]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4), sharex=True)

for i, avg in enumerate(averages):
    ax[0].loglog(a * (1-porosities[i]), avg['ka'][0], label=f'p = {porosities[i]:.4g}')
    ax[1].semilogx(a * (1-porosities[i]), avg['beta'], label=f'p = {porosities[i]:.4g}')
    
ax[0].set_xlim(1e-4, 1e3)
ax[0].set_ylim(1e-2, 1e1)
ax[0].set_ylabel(r'$\kappa_\mathrm{abs,1mm}$ [cm$^{2}$/g]')
ax[1].set_ylabel(r'$\beta$')
ax[1].set_ylim(0, 4.2)

for _ax in ax:
    _ax.legend().get_frame().set_alpha(0)
    _ax.grid()
    _ax.set_xlabel(r'$\left(a\cdot f\right)_\mathrm{max}$ [cm]')


fig.subplots_adjust(wspace=0.2);